# 따릉이 자료 전처리 및 Tableau를 통한 시각화

## 라이브러리 로드 및 구글드라이브 마운트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# matplotlib을 임포트 안하면 seaborn을 제대로 쓸 수 없다.

plt.rcParams["figure.figsize"] =(8, 5)

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 기본 데이터 전처리

### 대여 및 반납 정보 전처리

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/2112.csv",encoding="cp949")

In [ ]:
df.rename(
    columns={"대여 대여소번호" : "대여소번호",
             "대여 대여소명" : "대여소명"},
    inplace = True
)
df.drop(["자전거번호","대여거치대","반납거치대"], axis=1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704297 entries, 0 to 1704296
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   대여일시     object 
 1   대여소번호    int64  
 2   대여소명     object 
 3   반납일시     object 
 4   반납대여소번호  int64  
 5   반납대여소명   object 
 6   이용시간     int64  
 7   이용거리     float64
dtypes: float64(1), int64(3), object(4)
memory usage: 104.0+ MB


### 대여소 위치정보 전처리

In [ ]:
station = pd.read_csv("/content/drive/MyDrive/stat.csv",encoding="cp949")

In [ ]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2161 entries, 0 to 2160
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   대여소
번호      2154 non-null   float64
 1   보관소(대여소)명   2154 non-null   object 
 2   소재지(위치)     2155 non-null   object 
 3   Unnamed: 3  2155 non-null   object 
 4   Unnamed: 4  2043 non-null   object 
 5   Unnamed: 5  2043 non-null   object 
 6   설치
시기       2152 non-null   object 
 7   설치형태        1484 non-null   object 
 8   Unnamed: 8  673 non-null    object 
 9   운영
방식       2154 non-null   object 
dtypes: float64(1), object(9)
memory usage: 169.0+ KB


In [ ]:
station.head()

,대여소\n번호,보관소(대여소)명,소재지(위치),Unnamed: 3,Unnamed: 4,Unnamed: 5,설치\n시기,설치형태,Unnamed: 8,운영\n방식
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCD,QR,NaN
1,NaN,NaN,자치구,상세주소,위도,경도,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,거치\n대수,거치\n대수,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,301.0,경복궁역 7번출구 앞,종로구,서울특별시 종로구 사직로 지하130,37.575794,126.971451,2015-10-07,16,NaN,LCD


In [ ]:
station.rename(
    columns={"대여소\n번호" : "대여소번호",
             "보관소(대여소)명":"대여소명",
             "소재지(위치)" : "소재지",
             "Unnamed: 3" : "상세주소",
             "Unnamed: 4" : "위도",
             "Unnamed: 5" : "경도",
             "설치\n시기" : "설치시기",
             "Unnamed: 8" : "거치대수",
             "운영\n방식" : "운영방식"
             }, inplace = True
)
station = station.drop(range(0,4),axis=0).reset_index(drop=True) # 주의
delete_cols = ["상세주소","설치시기","설치형태","거치대수","운영방식","대여소명"]
station.drop(delete_cols, axis = 1, inplace = True)
del_indices = list(range(2154,2157))
station.drop(del_indices,axis=0, inplace = True)
station["대여소번호"] = station["대여소번호"].astype(int)

In [ ]:
station.tail()

,대여소번호,소재지,위도,경도
2149,1085,강동구,37.554569,127.145874
2150,1086,강동구,37.550598,127.145454
2151,1088,강동구,37.528122,127.119522
2152,1089,강동구,37.560600,127.146698
2153,1090,강동구,37.556789,127.166145


In [ ]:
station_1 = station.rename(
    columns={"위도" : "대여위도",
             "경도" : "대여경도",
             "소재지" : "대여소재지"
             }
)
station_1.tail()

,대여소번호,대여소재지,대여위도,대여경도
2149,1085,강동구,37.554569,127.145874
2150,1086,강동구,37.550598,127.145454
2151,1088,강동구,37.528122,127.119522
2152,1089,강동구,37.560600,127.146698
2153,1090,강동구,37.556789,127.166145


In [ ]:
station_2 = station.rename(
    columns={"대여소번호" : "반납대여소번호",
             "위도" : "반납위도",
             "경도" : "반납경도",
             "소재지" : "반납소재지"
             }
)
station_2.head()

,반납대여소번호,반납소재지,반납위도,반납경도
0,301,종로구,37.575794,126.971451
1,302,종로구,37.575947,126.974060
2,303,종로구,37.571770,126.974663
3,304,종로구,37.572113,126.977577
4,305,종로구,37.572582,126.978355


### 대여 정보 데이터 및 대여소 데이터 통합

In [ ]:
bike = pd.merge(df, station_1, on = "대여소번호", how = "inner")
bike = pd.merge(bike, station_2, on = "반납대여소번호", how = "inner")

In [ ]:
bike.head(1)

,대여일시,대여소번호,대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,대여소재지,대여위도,대여경도,반납소재지,반납위도,반납경도
0,2021-12-01 01:24:27,102,망원역 1번출구 앞,2021-12-01 01:35:24,3002,벽산상암스마트큐브,10,1260.0,마포구,37.555649,126.910629,마포구,37.565044,126.903603


### 1

In [ ]:
mask = (bike['대여소재지'] == "도봉구") & (bike["반납소재지"] == "광진구")
bike.loc[mask,["대여일시","대여소재지","대여소명","반납소재지","반납대여소명","이용시간","이용거리"]].sort_values(by = "이용시간", ascending =False)

,대여일시,대여소재지,대여소명,반납소재지,반납대여소명,이용시간,이용거리
788679,2021-12-15 21:23:56,도봉구,서울북부광역등기국,광진구,건대입구역 사거리(롯데백화점),145,24315.94
552756,2021-12-05 18:49:11,도봉구,도봉구청 옆(중랑천변),광진구,뚝섬유원지역 1번출구 앞,134,25900.00
552760,2021-12-04 10:41:27,도봉구,방학사거리 (봄마당 앞),광진구,뚝섬유원지역 1번출구 앞,126,21480.00
552758,2021-12-17 05:31:33,도봉구,도봉구청 옆(중랑천변),광진구,뚝섬유원지역 1번출구 앞,118,23351.36
552759,2021-12-05 19:05:43,도봉구,창동역 2번출구,광진구,뚝섬유원지역 1번출구 앞,118,19870.00
552757,2021-12-17 05:37:10,도봉구,도봉구청 옆(중랑천변),광진구,뚝섬유원지역 1번출구 앞,112,22993.33
552761,2021-12-08 19:39:02,도봉구,제일강산수산입구,광진구,뚝섬유원지역 1번출구 앞,106,21170.91
552762,2021-12-04 13:41:09,도봉구,창동19단지아파트 관리사무소앞,광진구,뚝섬유원지역 1번출구 앞,103,19480.00
1129511,2021-12-02 00:00:20,도봉구,창포원 남쪽 입구,광진구,원일교회,100,17710.00
656509,2021-12-03 16:37:55,도봉구,창동주공17단지 상가앞,광진구,군자역2번출구,94,0.00


### 2

In [ ]:
bike.tail()

,대여일시,대여소번호,대여소명,반납일시,반납대여소번호,반납대여소명,이용시간,이용거리,대여소재지,대여위도,대여경도,반납소재지,반납위도,반납경도
1396129,2021-12-15 15:56:30,2419,"봉은사역 1,2번 출구",2021-12-15 16:20:47,2304,신영 ROYAL PALACE 앞,24,3360.00,강남구,37.514309,127.059700,강남구,37.512527,127.035835
1396130,2021-12-11 01:03:35,2421,신논현역 4번출구,2021-12-11 01:17:21,2304,신영 ROYAL PALACE 앞,13,1580.00,강남구,37.504673,127.025795,강남구,37.512527,127.035835
1396131,2021-12-30 18:20:33,2421,신논현역 4번출구,2021-12-30 18:32:36,2304,신영 ROYAL PALACE 앞,12,2562.81,강남구,37.504673,127.025795,강남구,37.512527,127.035835
1396132,2021-12-22 18:20:04,2515,서초초등학교 후문,2021-12-22 18:40:42,2304,신영 ROYAL PALACE 앞,20,0.00,서초구,37.498928,127.024727,강남구,37.512527,127.035835
1396133,2021-12-28 18:07:57,2515,서초초등학교 후문,2021-12-28 18:26:45,2304,신영 ROYAL PALACE 앞,18,2669.79,서초구,37.498928,127.024727,강남구,37.512527,127.035835


## 데이터 내보내기

Tableau를 이용한 시각화 자료를 만들기 위해, 해당 데이터베이스를 csv 파일로 따로 저장한다.

In [ ]:
# bike.to_csv("bike_save.csv")

## Taleau를 이용한 지도 시각화

![따릉따릉따르릉](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FmBkM3%2Fbtrx6YJv1GD%2FQprOAkzlGlCUKEoZq0X77K%2Fimg.png)

- 대여소별 반납 장소를 표현한 지도이다.
- 전체 원의 크기는 대여소 별 대여 건수이며, 그 안에 반납 장소별로 사이즈가 구현되어있다.